# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 26 2022 1:07PM,247285,10,Enova-10701,Emerginnova,Released
1,Sep 26 2022 1:07PM,247285,10,Eme-104509,Emerginnova,Released
2,Sep 26 2022 1:07PM,247285,10,Enova-10702,Emerginnova,Released
3,Sep 26 2022 1:00PM,247284,10,SO94396,"Senseonics, Incorporated",Released
4,Sep 26 2022 1:00PM,247284,10,SO94397,"Senseonics, Incorporated",Released
5,Sep 26 2022 12:55PM,247281,19,N17781,"NAPP Technologies, LLC",Released
6,Sep 26 2022 12:55PM,247280,10,CTF0011428,"Citieffe, Inc.",Released
7,Sep 26 2022 12:49PM,247279,10,8576092,Yusen – 3D Matrix,Released
8,Sep 26 2022 12:44PM,247278,10,8576088,Yusen – 3D Matrix,Released
9,Sep 26 2022 12:37PM,247277,10,8576063,Yusen – 3D Matrix,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,247279,Released,1
27,247280,Released,1
28,247281,Released,1
29,247284,Released,2
30,247285,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247279,NaN,NaN,1.0
247280,NaN,NaN,1.0
247281,NaN,NaN,1.0
247284,NaN,NaN,2.0
247285,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247197,0.0,1.0,0.0
247199,0.0,1.0,0.0
247219,0.0,4.0,11.0
247220,0.0,0.0,3.0
247223,0.0,0.0,8.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247197,0,1,0
247199,0,1,0
247219,0,4,11
247220,0,0,3
247223,0,0,8


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247197,0,1,0
1,247199,0,1,0
2,247219,0,4,11
3,247220,0,0,3
4,247223,0,0,8


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247197,,1,
1,247199,,1,
2,247219,,4,11
3,247220,,,3
4,247223,,,8


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 26 2022 1:07PM,247285,10,Emerginnova
3,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated"
5,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC"
6,Sep 26 2022 12:55PM,247280,10,"Citieffe, Inc."
7,Sep 26 2022 12:49PM,247279,10,Yusen – 3D Matrix
8,Sep 26 2022 12:44PM,247278,10,Yusen – 3D Matrix
9,Sep 26 2022 12:37PM,247277,10,Yusen – 3D Matrix
10,Sep 26 2022 12:35PM,247276,19,"AdvaGen Pharma, Ltd"
11,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc."
12,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 26 2022 1:07PM,247285,10,Emerginnova,,,3
1,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",,,2
2,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",,,1
3,Sep 26 2022 12:55PM,247280,10,"Citieffe, Inc.",,,1
4,Sep 26 2022 12:49PM,247279,10,Yusen – 3D Matrix,,,1
5,Sep 26 2022 12:44PM,247278,10,Yusen – 3D Matrix,,,1
6,Sep 26 2022 12:37PM,247277,10,Yusen – 3D Matrix,,,1
7,Sep 26 2022 12:35PM,247276,19,"AdvaGen Pharma, Ltd",,1,
8,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",,,1
9,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:07PM,247285,10,Emerginnova,3,,
1,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",2,,
2,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",1,,
3,Sep 26 2022 12:55PM,247280,10,"Citieffe, Inc.",1,,
4,Sep 26 2022 12:49PM,247279,10,Yusen – 3D Matrix,1,,
5,Sep 26 2022 12:44PM,247278,10,Yusen – 3D Matrix,1,,
6,Sep 26 2022 12:37PM,247277,10,Yusen – 3D Matrix,1,,
7,Sep 26 2022 12:35PM,247276,19,"AdvaGen Pharma, Ltd",,1,
8,Sep 26 2022 11:59AM,247268,10,"Citieffe, Inc.",1,,
9,Sep 26 2022 11:58AM,247267,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:07PM,247285,10,Emerginnova,3,,
1,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",2,,
2,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",1,,
3,Sep 26 2022 12:55PM,247280,10,"Citieffe, Inc.",1,,
4,Sep 26 2022 12:49PM,247279,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:07PM,247285,10,Emerginnova,3.0,NaN,NaN
1,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",2.0,NaN,NaN
2,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",1.0,NaN,NaN
3,Sep 26 2022 12:55PM,247280,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Sep 26 2022 12:49PM,247279,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 26 2022 1:07PM,247285,10,Emerginnova,3.0,0.0,0.0
1,Sep 26 2022 1:00PM,247284,10,"Senseonics, Incorporated",2.0,0.0,0.0
2,Sep 26 2022 12:55PM,247281,19,"NAPP Technologies, LLC",1.0,0.0,0.0
3,Sep 26 2022 12:55PM,247280,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Sep 26 2022 12:49PM,247279,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4697913,116.0,6.0,1.0
16,494501,2.0,0.0,0.0
19,741815,2.0,1.0,0.0
20,247262,51.0,0.0,0.0
21,741642,0.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4697913,116.0,6.0,1.0
1,16,494501,2.0,0.0,0.0
2,19,741815,2.0,1.0,0.0
3,20,247262,51.0,0.0,0.0
4,21,741642,0.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,116.0,6.0,1.0
1,16,2.0,0.0,0.0
2,19,2.0,1.0,0.0
3,20,51.0,0.0,0.0
4,21,0.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,116.0
1,16,Released,2.0
2,19,Released,2.0
3,20,Released,51.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,1.0,0.0,3.0
Released,116.0,2.0,2.0,51.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,1.0,0.0,3.0
2,Released,116.0,2.0,2.0,51.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,1.0,0.0,3.0
2,Released,116.0,2.0,2.0,51.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()